In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [12]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_ak.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis =1) #968557
fips_check.head() #2663

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,13,13,5502308207,8/8/20,1084,PPP,"KAKIVIK ASSET MANAGEMENT, LLC",5015 BUSINESS PARK BLVD,ANCHORAGE,AK,...,Unanswered,N,9615884.57,6/11/21,54.0,"Professional, Scientific, and Technical Services",ALASKA DISTRICT OFFICE,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309
1,14,14,6110847106,4/14/20,1084,PPP,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,...,Unanswered,Y,7761324.81,7/13/21,81.0,Other Services (except Public Administration),ALASKA DISTRICT OFFICE,"7000 Uula St, BARROW, AK",71.297290,-156.728897
2,15,15,4539098204,8/6/20,1084,PPP,CORVUS AIRLINES INC,4700 Old International Airport Rd,Anchorage,AK,...,Unanswered,N,NaN,NaN,48.0,Transportation and Warehousing,ALASKA DISTRICT OFFICE,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718
3,16,16,5120868804,4/17/21,1084,PPP,HOPE COMMUNITY RESOURCES INC.,540 W Intl Airport Rd,Anchorage,AK,...,Unanswered,Y,NaN,NaN,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283
4,17,17,6650277102,4/14/20,1084,PPP,SOUTH PENINSULA HOSPITAL INC,4300 BARTLETT STREET,HOMER,AK,...,Unanswered,N,6598389.38,5/18/21,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793


In [14]:
columns = ['Unnamed: 0','LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [15]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #5

,Unnamed: 0,LoanNumber,full_add,Lat,Long
356,369,5048318601,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN
415,428,5085327008,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN
1852,1865,4637628408,"1706 Ship Ave, Anchorage, AK",NaN,NaN
2261,2274,5337318401,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN
2366,2379,9891427104,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN


In [16]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #2658

,Unnamed: 0,LoanNumber,full_add,Lat,Long
0,13,5502308207,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309
1,14,6110847106,"7000 Uula St, BARROW, AK",71.297290,-156.728897
2,15,4539098204,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718
3,16,5120868804,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283
4,17,6650277102,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/AK_fips_scraped.csv") 
fips_check_scraped.head()

In [103]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [ ]:
#pd.set_option('display.float_format', '{:.0f}'.format)
fips_check_scraped.iloc[:41109]

In [ ]:
fips_check_scraped_null = fips_check_scraped[fips_check_scraped["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null

<h4>FIPS Script

In [17]:
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

In [18]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('02')] #AKshrink ss
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [19]:
geo_id_list =geo_id 

In [20]:
print(len(geo_id_list))

2658


In [22]:
fips_fill = fips_check_notna
fips_fill

,Unnamed: 0,LoanNumber,full_add,Lat,Long
0,13,5502308207,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309
1,14,6110847106,"7000 Uula St, BARROW, AK",71.297290,-156.728897
2,15,4539098204,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718
3,16,5120868804,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283
4,17,6650277102,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793
...,...,...,...,...,...
2658,2671,7365917001,"188 W NORTHERN LIGHTS BLVD, SUITE 1020, ANCHOR...",61.194692,-149.886020
2659,2672,7400167008,"1119 E. 70TH AVE, ANCHORAGE, AK",61.157184,-149.861692
2660,2673,7483247209,"425 West 58th Ave, anchorage, AK",61.169044,-149.889214
2661,2674,8414228309,"936 E Westpoint Dr, Wasilla, AK",61.582021,-149.427018


In [23]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

<ipython-input-23-08e9c917fe2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,Unnamed: 0,LoanNumber,full_add,Lat,Long,FIPS_z
0,13,5502308207,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,[020200019003022]
1,14,6110847106,"7000 Uula St, BARROW, AK",71.297290,-156.728897,[021850001001069]
2,15,4539098204,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,[020200023011009]
3,16,5120868804,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,[020200025012000]
4,17,6650277102,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,[021220010002001]
...,...,...,...,...,...,...
2658,2671,7365917001,"188 W NORTHERN LIGHTS BLVD, SUITE 1020, ANCHOR...",61.194692,-149.886020,[020200019001002]
2659,2672,7400167008,"1119 E. 70TH AVE, ANCHORAGE, AK",61.157184,-149.861692,[020200025022018]
2660,2673,7483247209,"425 West 58th Ave, anchorage, AK",61.169044,-149.889214,[020200025012003]
2661,2674,8414228309,"936 E Westpoint Dr, Wasilla, AK",61.582021,-149.427018,[021700008003022]


In [26]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

<ipython-input-26-e19831f223ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,Unnamed: 0,LoanNumber,full_add,Lat,Long,FIPS_z
0,13,5502308207,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,020200019003022
1,14,6110847106,"7000 Uula St, BARROW, AK",71.297290,-156.728897,021850001001069
2,15,4539098204,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,020200023011009
3,16,5120868804,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,020200025012000
4,17,6650277102,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,021220010002001
...,...,...,...,...,...,...
2658,2671,7365917001,"188 W NORTHERN LIGHTS BLVD, SUITE 1020, ANCHOR...",61.194692,-149.886020,020200019001002
2659,2672,7400167008,"1119 E. 70TH AVE, ANCHORAGE, AK",61.157184,-149.861692,020200025022018
2660,2673,7483247209,"425 West 58th Ave, anchorage, AK",61.169044,-149.889214,020200025012003
2661,2674,8414228309,"936 E Westpoint Dr, Wasilla, AK",61.582021,-149.427018,021700008003022


In [31]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()]

,Unnamed: 0,LoanNumber,full_add,Lat,Long,FIPS_z
392,405,2725348408,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
1095,1108,4188937110,"525 SECOND STREET, SKAGWAY, AK",40.670782,-73.976908,NaN
1602,1615,9864858410,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
1692,1705,5628247103,"6100 A Street, ANCHORAGE, AK",61.165676,-149.882046,NaN
1848,1861,9635677110,"8840 Old Seward Hwy Ste E, ANCHORAGE, AK",61.140366,-149.865092,NaN
2504,2517,2484417102,"2767 Sherwood Lane Unit C, JUNEAU, AK",58.370927,-134.617716,NaN


In [32]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,5502308207,020200019003022
1,6110847106,021850001001069
2,4539098204,020200023011009
3,5120868804,020200025012000
4,6650277102,021220010002001
...,...,...
2658,7365917001,020200019001002
2659,7400167008,020200025022018
2660,7483247209,020200025012003
2661,8414228309,021700008003022


In [35]:
fips_check = pd.read_csv("data/state_data/geo/geo_ak.csv")
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) #968557
fips_check.head() #2663

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,5502308207,8/8/20,1084,PPP,"KAKIVIK ASSET MANAGEMENT, LLC",5015 BUSINESS PARK BLVD,ANCHORAGE,AK,99503-7146,7/22/21,...,Unanswered,NaN,9615884.57,6/11/21,54.0,"Professional, Scientific, and Technical Services",ALASKA DISTRICT OFFICE,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309
1,6110847106,4/14/20,1084,PPP,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723,8/21/21,...,Unanswered,Y,7761324.81,7/13/21,81.0,Other Services (except Public Administration),ALASKA DISTRICT OFFICE,"7000 Uula St, BARROW, AK",71.297290,-156.728897
2,4539098204,8/6/20,1084,PPP,CORVUS AIRLINES INC,4700 Old International Airport Rd,Anchorage,AK,99502-1130,NaN,...,Unanswered,NaN,NaN,NaN,48.0,Transportation and Warehousing,ALASKA DISTRICT OFFICE,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718
3,5120868804,4/17/21,1084,PPP,HOPE COMMUNITY RESOURCES INC.,540 W Intl Airport Rd,Anchorage,AK,99518-1105,NaN,...,Unanswered,Y,NaN,NaN,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283
4,6650277102,4/14/20,1084,PPP,SOUTH PENINSULA HOSPITAL INC,4300 BARTLETT STREET,HOMER,AK,99603,6/9/21,...,Unanswered,NaN,6598389.38,5/18/21,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793


In [36]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,5502308207,8/8/20,1084,PPP,"KAKIVIK ASSET MANAGEMENT, LLC",5015 BUSINESS PARK BLVD,ANCHORAGE,AK,99503-7146,7/22/21,...,NaN,9615884.57,6/11/21,54.0,"Professional, Scientific, and Technical Services",ALASKA DISTRICT OFFICE,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,020200019003022
1,6110847106,4/14/20,1084,PPP,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723,8/21/21,...,Y,7761324.81,7/13/21,81.0,Other Services (except Public Administration),ALASKA DISTRICT OFFICE,"7000 Uula St, BARROW, AK",71.297290,-156.728897,021850001001069
2,4539098204,8/6/20,1084,PPP,CORVUS AIRLINES INC,4700 Old International Airport Rd,Anchorage,AK,99502-1130,NaN,...,NaN,NaN,NaN,48.0,Transportation and Warehousing,ALASKA DISTRICT OFFICE,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,020200023011009
3,5120868804,4/17/21,1084,PPP,HOPE COMMUNITY RESOURCES INC.,540 W Intl Airport Rd,Anchorage,AK,99518-1105,NaN,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,020200025012000
4,6650277102,4/14/20,1084,PPP,SOUTH PENINSULA HOSPITAL INC,4300 BARTLETT STREET,HOMER,AK,99603,6/9/21,...,NaN,6598389.38,5/18/21,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,021220010002001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2658,7365917001,4/7/20,1084,PPP,"BANKSTON GRONNING BRECHT, PC","188 W NORTHERN LIGHTS BLVD, SUITE 1020",ANCHORAGE,AK,99503-3902,8/20/21,...,NaN,151890.41,7/22/21,54.0,"Professional, Scientific, and Technical Services",ALASKA DISTRICT OFFICE,"188 W NORTHERN LIGHTS BLVD, SUITE 1020, ANCHOR...",61.194692,-149.886020,020200019001002
2659,7400167008,4/7/20,1084,PPP,"PHILLIPS SCALES, LLC",1119 E. 70TH AVE,ANCHORAGE,AK,99518-2353,4/20/21,...,NaN,151356.16,3/18/21,33.0,Manufacturing,ALASKA DISTRICT OFFICE,"1119 E. 70TH AVE, ANCHORAGE, AK",61.157184,-149.861692,020200025022018
2660,7483247209,4/28/20,1084,PPP,ROYAL BUSINESS SYSTEMS,425 West 58th Ave,anchorage,AK,99518-1625,9/29/21,...,NaN,151878.08,8/16/21,45.0,Retail Trade,ALASKA DISTRICT OFFICE,"425 West 58th Ave, anchorage, AK",61.169044,-149.889214,020200025012003
2661,8414228309,1/29/21,1084,PPS,ALASKA COMPREHENSIVE DENTAL CENTER INC,936 E Westpoint Dr,Wasilla,AK,99654-7137,NaN,...,NaN,151150.68,11/12/21,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"936 E Westpoint Dr, Wasilla, AK",61.582021,-149.427018,021700008003022


In [39]:
fips_AK = fips_merge

In [4]:
geo_AK_fips.to_csv("data/state_data/geo/geo_fips/AK_fips_scraped.csv") 

In [5]:
geo_AK_fips = pd.read_csv("data/state_data/geo/geo_fips/AK_fips_scraped.csv") 
geo_AK_fips 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z,LongName,IndustrySubsector
0,0,0,0,0,5502308207,8/8/20,1084,PPP,"KAKIVIK ASSET MANAGEMENT, LLC",5015 BUSINESS PARK BLVD,...,6/11/21,54.0,"Professional, Scientific, and Technical Services",ALASKA DISTRICT OFFICE,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13,"All Other Professional, Scientific, and Techni...","5419 Other Professional, Scientific, and Techn..."
1,1,1,1,1,6110847106,4/14/20,1084,PPP,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,...,7/13/21,81.0,Other Services (except Public Administration),ALASKA DISTRICT OFFICE,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13,Professional Organizations,"8139 Business, Professional, Labor, Political,..."
2,2,2,2,2,4539098204,8/6/20,1084,PPP,CORVUS AIRLINES INC,4700 Old International Airport Rd,...,NaN,48.0,Transportation and Warehousing,ALASKA DISTRICT OFFICE,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13,Scheduled Passenger Air Transportation,4811 Scheduled Air Transportation
3,3,3,3,3,5120868804,4/17/21,1084,PPP,HOPE COMMUNITY RESOURCES INC.,540 W Intl Airport Rd,...,NaN,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13,Services for the Elderly and Persons with Disa...,6241 Individual and Family Services
4,4,4,4,4,6650277102,4/14/20,1084,PPP,SOUTH PENINSULA HOSPITAL INC,4300 BARTLETT STREET,...,5/18/21,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13,General Medical and Surgical Hospitals,6221 General Medical and Surgical Hospitals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2658,2658,2658,2658,2658,7365917001,4/7/20,1084,PPP,"BANKSTON GRONNING BRECHT, PC","188 W NORTHERN LIGHTS BLVD, SUITE 1020",...,7/22/21,54.0,"Professional, Scientific, and Technical Services",ALASKA DISTRICT OFFICE,"188 W NORTHERN LIGHTS BLVD, SUITE 1020, ANCHOR...",61.194692,-149.886020,2.020002e+13,Offices of Lawyers,5411 Legal Services
2659,2659,2659,2659,2659,7400167008,4/7/20,1084,PPP,"PHILLIPS SCALES, LLC",1119 E. 70TH AVE,...,3/18/21,33.0,Manufacturing,ALASKA DISTRICT OFFICE,"1119 E. 70TH AVE, ANCHORAGE, AK",61.157184,-149.861692,2.020003e+13,All Other Miscellaneous Fabricated Metal Produ...,3329 Other Fabricated Metal Product Manufacturing
2660,2660,2660,2660,2660,7483247209,4/28/20,1084,PPP,ROYAL BUSINESS SYSTEMS,425 West 58th Ave,...,8/16/21,45.0,Retail Trade,ALASKA DISTRICT OFFICE,"425 West 58th Ave, anchorage, AK",61.169044,-149.889214,2.020003e+13,Office Supplies and Stationery Stores,"4532 Office Supplies, Stationery, and Gift Sto..."
2661,2661,2661,2661,2661,8414228309,1/29/21,1084,PPS,ALASKA COMPREHENSIVE DENTAL CENTER INC,936 E Westpoint Dr,...,11/12/21,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"936 E Westpoint Dr, Wasilla, AK",61.582021,-149.427018,2.170001e+13,Offices of Dentists,6212 Offices of Dentists


In [3]:
#fill in missing Lat, Long, FIPS
geo_AK_fips.loc[geo_AK_fips["FIPS_z"].isnull()]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
356,356,356,356,5048318601,3/20/21,1084,PPS,DELTA INDUSTRIAL SERVICES INC.,1229 RICHARDSON HWY,DELTA JUNCTION,...,N,NaN,NaN,55.0,Management of Companies and Enterprises,ALASKA DISTRICT OFFICE,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN,NaN
392,392,392,392,2725348408,2/4/21,1084,PPS,ALASKA ICEFIELD EXPEDITIONS INC,525 Second Street,Skagway,...,N,NaN,NaN,71.0,"Arts, Entertainment, and Recreation",ALASKA DISTRICT OFFICE,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
415,415,415,415,5085327008,4/5/20,1084,PPP,"DELTA INDUSTRIAL SERVICES, INC.",1229 Richardson Hwy,DELTA JUNCTION,...,N,744923.33,2/22/21,55.0,Management of Companies and Enterprises,ALASKA DISTRICT OFFICE,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN,NaN
1095,1095,1095,1095,4188937110,4/13/20,1084,PPP,"ALASKA ICEFIELD EXPEDITIONS, INC.",525 SECOND STREET,SKAGWAY,...,N,339009.00,6/4/21,71.0,"Arts, Entertainment, and Recreation",ALASKA DISTRICT OFFICE,"525 SECOND STREET, SKAGWAY, AK",40.670782,-73.976908,NaN
1602,1602,1602,1602,9864858410,2/18/21,1084,PPS,GOLD RUSH DOG TOURS LLC,525 Second Street,Skagway,...,N,NaN,NaN,71.0,"Arts, Entertainment, and Recreation",ALASKA DISTRICT OFFICE,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
1692,1692,1692,1692,5628247103,4/13/20,1084,PPP,"LAST FRONTIER MECHANICAL, LLC",6100 A Street,ANCHORAGE,...,N,183471.94,5/21/21,23.0,Construction,ALASKA DISTRICT OFFICE,"6100 A Street, ANCHORAGE, AK",61.165676,-149.882046,NaN
1848,1848,1848,1848,9635677110,4/15/20,1084,PPP,"BETTER HEALTH PAIN AND WELNESS CENTER, LLC",8840 Old Seward Hwy Ste E,ANCHORAGE,...,N,211126.01,8/16/21,62.0,Health Care and Social Assistance,ALASKA DISTRICT OFFICE,"8840 Old Seward Hwy Ste E, ANCHORAGE, AK",61.140366,-149.865092,NaN
1852,1852,1852,1852,4637628408,2/6/21,1084,PPS,MCKINLEY FIRE PROTECTION LLC,1706 Ship Ave,Anchorage,...,N,NaN,NaN,23.0,Construction,ALASKA DISTRICT OFFICE,"1706 Ship Ave, Anchorage, AK",NaN,NaN,NaN
2261,2261,2261,2261,5337318401,2/8/21,1084,PPS,THE ALASKA CENTRE FOR CREATIVE RENEWAL LLC,46877 STILLPOINT TRL,HALIBUT COVE,...,N,NaN,NaN,72.0,Accomodation and Food Services,ALASKA DISTRICT OFFICE,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN,NaN
2366,2366,2366,2366,9891427104,4/15/20,1084,PPP,MCKINLEY FIRE PROTECTION,1706 SHIP AVENUE,ANCHORAGE,...,N,168353.89,2/19/21,23.0,Construction,ALASKA DISTRICT OFFICE,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN,NaN
